# Record Linkage Example Code

In [1]:
scripts_folder = "/Users/descobarsalce/Library/CloudStorage/Dropbox/CV/GITHUB/"
data_folder = "/Users/descobarsalce/Library/CloudStorage/Dropbox/Philanthropy/"

In [2]:
import pandas as pd 
import numpy as np 
import pickle
import os

This script finds matching names in web-scrapped data with inconsistent names and administrative data to track instituions over time and use both set of outcomes. We start by loading the web data that has already been processed from the html files and saved in the csv file "all_grants_transactions":

## Load datasets:

In [3]:
# Set data directory:
os.chdir(data_folder)

First we load the web scraped data (all_transactions) which contains non-homogeneous names for institutions (self reported by contributors).

In [4]:
# Load scraped data for preprocessing:
all_data_file = 'data_processed/all_grants_transactions.csv'
column_names = ['EIN', 'Foundation', 'Giver_City', 'Giver_State', 'Recipient_Name', 'Recipient_City', 
                'Recipient_State', 'Giving_Category', 'Amount', 'Year', 'Grant_Description', 'Contact', 
                'Telephone']
# I need to load all data as objects before before pre-processing they all contain at least some incorrect non-numerical entries.
data_type = {'EIN': 'object', 'Foundation': 'object', 'Giver_City': 'object',   'Giver_State': 'object',
             'Recipient_Name': 'object', 'Recipient_City': 'object', 'Recipient_State': 'object', 
             'Giving_Category': 'object', 'Amount': 'object', 'Year': 'object',  'Grant_Description': 'object',
             'Contact': 'object', 'Telephone': 'object','id_variable': 'object'}

all_transactions = pd.read_csv(all_data_file, encoding="ISO-8859–1", index_col=False, on_bad_lines="skip", 
                               header=None, names=column_names, dtype=data_type)
# For the example on this code I wont use all data so we'll drop some of the variables:
all_transactions.drop(['Giver_City', 'Recipient_City', 'Contact', 'Amount', 'Telephone'], axis=1, inplace=True)

In [5]:
# Pre-process to keep only the relevant entries from web-scraped data
all_transactions.Year = pd.to_numeric(all_transactions.Year, errors='coerce')
all_transactions = all_transactions[(all_transactions.Year>=2000) & (all_transactions.Year<2018)]
all_transactions = all_transactions[all_transactions['Giving_Category']=='Education']

# Complement with other data sources for year 2018 (incomplete on web-scraped data, but not available for 
# previous year)
grants_Open990 = pd.read_pickle('data_processed/grants_2018_Open990')

# Merge both data sources:
all_transactions = pd.concat([all_transactions, grants_Open990], ignore_index=True)

# Homogeneize states:
all_transactions['Recipient_State'] = all_transactions['Recipient_State'].str.upper()
all_transactions['Giver_State'] = all_transactions['Giver_State'].str.upper()

# When receiver state info is not available I will input the one from the donors:
all_transactions['Recipient_State'] = np.where(all_transactions['Recipient_State'].isnull(), 
                                               all_transactions['Giver_State'].str.upper(), 
                                               all_transactions['Recipient_State'])
# Load states abbreviations:
with open('us_state_abbrev.pkl', 'rb') as f:
    us_state_abbrev = pickle.load(f)    
# I can only use data with some information about their states so we filter the rest:
all_transactions = all_transactions[all_transactions['Recipient_State'].isin(set(us_state_abbrev.values()))]

# Create an index for tracking and merging data later:
all_transactions['id_variable'] = all_transactions.index.astype('float64')

# Keep pre-processed data for later use:
all_transactions.to_pickle('data_processed/TRANSACTIONS_processed_python')

Now that the web-scrapped data is processed we turn into the administrative data from NCES to find the correct institution names:

In [6]:
# Load administrative data to make it uniform with scraped data and merge:
nces_data = pd.read_csv('data_processed/NCES_Processed.csv',  encoding="ISO-8859–1")

# Keep relevant variables for matching:
nces_data = nces_data[['instname_NCES', 'unitid', 'year', 'stabbr']]

In [7]:
# Donations data is yearly while Admin data is created biyearly, so the following fills in panel missing years:
nces_data['max_year'] = nces_data.groupby(['unitid'])['year'].transform("max")
nces_data['min_year'] = nces_data.groupby(['unitid'])['year'].transform("min")
mux = pd.MultiIndex.from_product([nces_data['unitid'].unique(), 
                                  np.arange(nces_data['year'].min(), nces_data['year'].max() + 1)],
                                  names=['unitid','year'])
nces_data =  nces_data.set_index(['unitid','year']).reindex(mux).reset_index()

# Detect whether an institution was active in certain year to discard mismatches:
nces_data['max_year'] = nces_data.groupby(['unitid'])['max_year'].transform("max")
nces_data['min_year'] = nces_data.groupby(['unitid'])['min_year'].transform("min")

# Need to fill in institution names/states for those units created when expanding the panel:
nces_data['instname_NCES'] = nces_data.groupby(['unitid'])['instname_NCES'].transform("first")
nces_data['stabbr'] = nces_data.groupby(['unitid'])['stabbr'].transform("first")
nces_data = nces_data[(nces_data.year>=nces_data.min_year) & (nces_data.year<=nces_data.max_year)]

# Keep pre-processed data for later use:
nces_data.to_pickle('data_processed/NCES_processed_python')

## Datasets example:

The following lines show a sample of each dataset. Transactions data uses the EIN tax ID identifier of contributing foundations, while NCES data uses the UNITID from the National Center for Educational Statistics.

In [8]:
print(all_transactions.shape)
all_transactions.sample(5)

(3383231, 11)


,EIN,Foundation,Giver_State,Recipient_Name,Recipient_State,Giving_Category,Year,Grant_Description,Amount,Recipient_City,id_variable
2301844,521328375,NORFOLK SOUTHERN FOUNDATION,VA,AUBURN UNIVERSITY FOUNDATION,AL,Education,2015.0,Matching gift,NaN,NaN,2301844.0
834731,526042568,MASONIC FOUNDATION OF THE DISTRICT OF COLUMBIA,DC,WINSTON-SALEM STATE UNIVERSITY,NC,Education,2016.0,Student scholarship,NaN,NaN,834731.0
3373098,200443423,NaN,VT,united way,VT,NaN,2018.0,operating funds,5000.0,middlebury,3373098.0
10936,113692807,SEAWORLD & BUSCH GARDENS CONSERVATION FUND,FL,ROGERS WILLIAMS UNIVERSITY,RI,Education,2015.0,GENERAL,NaN,NaN,10936.0
2587636,010498551,JOHN T GORMAN FOUNDATION,ME,"CARSEY INSTITUTE, UNIVERSITY OF NEW HAMPSHIRE",NH,Education,2014.0,To produce a data report about the well-being ...,NaN,NaN,2587636.0


In [9]:
print(nces_data.shape)
nces_data.sample(5)

(137258, 6)


,unitid,year,instname_NCES,stabbr,max_year,min_year
58382,173726,2014,COSMETOLOGY CAREERS UNLIMITED-HIBBING,MN,2016.0,2000.0
88053,206011,2007,TERRA STATE COMMUNITY COLLEGE,OH,2018.0,2000.0
79737,197814,2013,COLLEGE OF THE ALBEMARLE,NC,2018.0,2000.0
139392,382504,2008,KINGS COLLEGE,NC,2018.0,2000.0
26698,136491,2003,PINELLAS TECHNICAL EDUCATION CENTER-CLEARWATER,FL,2018.0,2000.0


# Record Linkage Function Usage:

Now that both datasets are loaded we can do to the actually relevant task of doing record linkage. The function is designed so that we need to load two dataframes with the corresponding names

In [10]:
os.chdir(scripts_folder)
from matching_function import RecordLinkage

In [11]:
# Name of the web-scraped data and the names variable:
df_messy = all_transactions
df_messy_names = 'Recipient_Name'
# Name of the administrative data and the names variable:
df_clean = nces_data
df_clean_names = "instname_NCES"
# N-grams to use in comparisons. Must provide a list of the length of the n-grams to be used. 
# All of the specified n-grams are used jointly when comparing cosine similarity. 
ngram_sizes = [3,4]
# Beside the n-grams, we can also include each word entirely.
include_full_words=True

# Create record linkage instance for web scrapped and admin data:
RecordLinkerUniversities = RecordLinkage(df_messy, df_messy_names, df_clean, df_clean_names)

Before attempting any record linkage we need to homogeneize the name of the strings. The output will be stored in the class internal variables and will clean the names on both datasets. If the datasets are long this may take some time.

In [12]:
# Run clean string to make variables uniform and eliminate things like caps, signs, etc.
RecordLinkerUniversities.clean_all_strings()

The record linkage performs two different operations:
1. Names in the web scrapped data are sometimes inconsistent within the same dataset depending on the contributing foundation, so we make them uniform by deduplicating entries. For example, names in the scraped data can appear with different variations (e.g. UChicago, Univ of Chicago, University of Chicago, etc). For this application, the admin dataset does not need to go through this process.
2. Names in both dataset are matches in the next step between datasets.

In [13]:
# Threshold affects the number/quality of matches. I'll just keep the same 0.8 because I'll use a ML model later to decide on a threshold.
# If no ML model will be used later it may be better to choose a lower number of neighbors (ntop) and a higher 
# lower bound. Mtches below this degree of similarity will be discarded, hence improving matches precision but 
# diminishing recall (percentage of matches that are identified).
df_messy_duplicates = RecordLinkerUniversities.deduplication(ntop=3, lower_bound=0.8, max_matches=None, 
                                                             n_grams=ngram_sizes, full_words=include_full_words)

SELFTIMED: 3305.600443840027


In [14]:
# Record linkage between clean and messy names:
# Finding 3 most similar neighbors that will then be pruned, if needed, using ML methods.

# Similarly to the code above, we can tune number of neighbors and maximum acceptable differences more strictly 
# if we are not going to perform a second pruning stage.
linked_data = RecordLinkerUniversities.record_linkage(number_neighbors=3, max_difference=0.7, n_grams=ngram_sizes, 
                                                      full_words=include_full_words)

Vectorizing the data - this could take a few minutes for large datasets...
Vectorizing completed...
Getting nearest neighbors...
Finding matches...
Building data frame...
Done


In [15]:
df_messy_duplicates.sample(20)

,left_side,right_side,similarity
746508,southside baseball association,southside baseball association,1.000000
207612,presbyterian childrens services,presbyterian childrens home and services,0.816437
288963,texas a and university kingsville,texas a and university kingsville,1.000000
255806,teach every nation,teach every nation,1.000000
617781,habitat for humanity international,international habitat for humanity,0.939343
470938,friends of the jerusalem academy of music,friends of the jerusalem academy of music,1.000000
540215,national organization for the advancement of ...,national organization for the advancement of ...,0.898427
205148,athletics for education,student athletics for education,0.849895
610200,acumen,acumen foundation,0.871156
385654,hack augusta,hack augusta,1.000000


In [16]:
linked_data.sample(20)

,differences_index,clean_name,messy_name
17369,0.69,solano county community college district,solano community college
3149,0.61,edgewood college,edgewood college school of nursin
5490,0.59,university of massachusetts lowell,university of massachusetts lrb
8703,0.59,stanford university,stanford university clsac
25143,0.55,pennsylvania state university penn state beaver,pennsylvania state university beaver
12062,0.36,long island university c w post campus,long island university c w post college
4894,0.69,yeshiva university,yeshiva university stem college
1008,0.69,notre dame college,notre dame prep
19328,0.55,seminary of the immaculate conception,saint mary of the immaculate conception
4526,0.65,university of notre dame,university of notre dame engineering
